<H1>Create a simple cantilever line beam model</H1>


In [62]:
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.21.1")

In [63]:
# Input Params
length = 5.0 #m		     Length of beam
point_load = 10.0  #kN   Concentrated load on cantilever tip
model_name = "Cantilever Line Beam" 

When interacting with Modeller externally a large speed up can be achived by setting the model to be invisible. Take care to restore the visibility and watch out for exceptions in the code that can leave Modeller running in the background.

In [64]:
if 0: # To speed up interaction with Modeller from python set the app invisible (toggle this to true)
    lusas.setVisible(False)

In [65]:
# Create a new model
db = lusas.newDatabase("Structural", f"{model_name}.mdl")
# Set the vertical axis
db.setLogicalUpAxis("Z")
# Set the unit system
db.setModelUnits("kN,m,t,s,C")

In [66]:
''' *** Create a line ***'''
# Get the geometry data object
geomData = lusas.geometryData()
# Set the defaults
geomData.setAllDefaults()
# Set the coordinates of the first point
geomData.addCoords(0, 0, 0)
# Set the coordinates of the second point
geomData.addCoords(length, 0, 0)
# Create the line object from the geomety data
objs = db.createLine(geomData) # Modeller returns an object set containing all the lines created, in this case we only have 1.
# Get the lines
lines = objs.getObjects("Lines")
# Get the single line from the returned array
beamLine = lines[0]

In [67]:
''' *** Create a mesh attribute ***'''
meshAttr = db.createMeshLine("Beam Mesh")
# Set the element type and size of elements BMI21 = 2Node thick beam elements, 1 = 1m length
meshAttr.setSize("BMI21", 1)

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 21.1.IFMeshLine instance at 0x2956302986240>

In [68]:
''' *** Create a geometric attribute ***'''
geomAttr = db.createGeometricLine("Beam Geometry")
# Set the element type
geomAttr.setValue("elementType", "3D Thick Beam")
# Set the beam section properties for the section library
geomAttr.setFromLibrary("UK Sections", "Universal Beams (BS4)", "406x178x74kg UB", 0, 0, 0)

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 21.1.IFGeometricLine instance at 0x2956303423376>

In [69]:
''' *** Create a material attribute ***'''
materialAttr = db.createIsotropicMaterial("Steel", 209000000.0, 0.3, 7.8)

In [70]:
''' *** Create a support attribute (fixed) *** '''
fixedSupport = db.createSupportStructural("Fixed")
# set the freedoms F=Free, R=Restrained
fixedSupport.setStructural("R", "R", "R", "R", "R", "R", "F", "F", "F")

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 21.1.IFSupportStructural instance at 0x2956303420928>

In [71]:
''' *** Create a point load attribute *** '''
loadAttr = db.createLoadingConcentrated("Point load")
loadAttr.setConcentrated(0.0, 0.0, -point_load)

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 21.1.IFLoadingConcentrated instance at 0x2956302984992>

In [72]:
''' *** Assign the attributes to the geometry *** '''
# get the assignment object
assignment = lusas.assignment()
# set the defaults
assignment.setAllDefaults()

# Assign the mesh
meshAttr.assignTo(beamLine, assignment)
# Assign the geometry
geomAttr.assignTo(beamLine, assignment)
# Assign the material
materialAttr.assignTo(beamLine, assignment)
# Assign the loading
loadAttr.assignTo(beamLine.getEndPoint(), assignment)

''' Assign the supports to the points of the line '''
# Assign the fixed support to the first point
fixedSupport.assignTo(beamLine.getStartPoint(), assignment)


<win32com.gen_py.LUSAS Modeller ActiveX Script Language 21.1.IFObjectSet instance at 0x2956303062640>

Create an eigenvalue frequency analysis

In [73]:
# Create a new analysis without an initial loadcase
analysis = db.createAnalysisStructural("Eigenvalue Frequency Analysis", False)
# Create a loadcase in the analysis
first_loadcase = db.createLoadcase("Eig Freq", analysis.getName())
# Set the loadcase controls to be a frequency analysis searching for the minimum 5 natural frequencies
first_loadcase.setEigenvalueMaxMinControl("Frequency", "Minimum", 5)


<win32com.gen_py.LUSAS Modeller ActiveX Script Language 21.1.IFLoadcase instance at 0x2956302988112>

Create a second eigenvalue frequency analysis with the point load considered as mass

In [74]:
# Create a new analysis without an initial loadcase
analysis = db.createAnalysisStructural("Eigenvalue Frequency Analysis With Load", False)
# Create a loadcase in the analysis
first_loadcase = db.createLoadcase("Eig Freq with load", analysis.getName())
# Set the loadcase controls to be a frequency analysis searching for the minimum 5 natural frequencies
first_loadcase.setEigenvalueMaxMinControl("Frequency", "Minimum", 5)
# consider the assigned loading as mass in the eigenvalue analysis
first_loadcase.getEigenvalueControl().setValue("loadToMass", True)
# Create the assignment data to target the new loadcase
assignment = lusas.assignment().setAllDefaults().setLoadset(first_loadcase)
# Assign the loading
loadAttr.assignTo(beamLine.getEndPoint(), assignment)


<win32com.gen_py.LUSAS Modeller ActiveX Script Language 21.1.IFObjectSet instance at 0x2956303428608>